# Car Detection with YOLOv8 (Custom Kaggle Dataset)

- Framework: Ultralytics YOLOv8 (nano)
- Task: Single-class car detection
- Dataset: "Car Detection and Tracking Dataset" from Kaggle
- Environment: Google Colab + KaggleHub

This notebook downloads a custom dataset from Kaggle, trains a YOLOv8n model to detect cars, and runs inference on sample images.


In [ ]:
# Install the ultralytics library that contains YOLOv8
!pip install ultralytics
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import cv2

In [ ]:
# Load a pre-trained YOLOv8 nano model (smallest version for fast training/testing)
model = YOLO("yolov8n.pt")

In [ ]:
# Upload your kaggle.json file to access Kaggle datasets
from google.colab import files
files.upload()

In [ ]:
# To download any dataset from Kaggle
# Create a hidden folder called `.kaggle` in the user's home directory
!mkdir -p ~/.kaggle
# Copy the kaggle.json file (API key) into the `.kaggle` folder
!cp kaggle-2.json ~/.kaggle/
# Change file permissions so only the user can read/write the kaggle.json file
!chmod 600 ~/.kaggle/kaggle-2.json

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amitkumargurjar/car-detection-and-tracking-dataset")

print("Path to dataset files:", path)

In [ ]:
extract_path = path
# Print path and show all files inside the dataset folder
print("Extracted files are located here:", extract_path)
print("Contents:", os.listdir(extract_path))

In [ ]:
# Return all files in this path
os.listdir(extract_path)

In [ ]:
yaml_content = """
train: /root/.cache/kagglehub/datasets/amitkumargurjar/car-detection-and-tracking-dataset/versions/2/archive/train
val: /root/.cache/kagglehub/datasets/amitkumargurjar/car-detection-and-tracking-dataset/versions/2/archive/val
nc: 1  # number of classes (1 = car)
names: ['car']
"""
# Create and save the YAML file in your working directory
with open('car_dataset.yaml', 'w') as f:
    f.write(yaml_content)
# Confirm the YAML file was created successfully
print("car_dataset.yaml created successfully.")
print(os.listdir('.'))

In [ ]:
dataset_yaml = "/content/car_dataset.yaml"
dataset_yaml

In [ ]:
# the original dataset path if it were downloaded directly from Kaggle
extract_path = "/kaggle/input/car-detection-and-tracking-dataset"
# paths to the training and validation folders after using kagglehub to download the dataset
train_data_path = os.path.join(extract_path, 'archive', 'train')
val_data_path = os.path.join(extract_path, 'archive', 'val')
# This is the YAML content YOLO uses to locate the images and know how many classes there are
yaml_content = f"""
train: {train_data_path}
val: {val_data_path}
nc: 1
names: ['car']
"""
with open('car_dataset.yaml', 'w') as f:
    f.write(yaml_content)
dataset_yaml = "/content/car_dataset.yaml"

# Using YOLOv8n in the model
model = YOLO("yolov8n.pt")
model.train(data=dataset_yaml, epochs=13, imgsz=640, batch=8, project="car_detection", name="yolov8_car")

In [ ]:
train_img = "/content/car_detection/yolov8_car2/train_batch152.jpg"
results = model.predict(train_img, save=True)
if results:
    results[0].show()  # Display the first image with bounding boxes

In [ ]:
# Import glob to search for images by file pattern
import glob
test_images = glob.glob("/content/car_detection/yolov8_car3/BoxPR_curve.png")
for img in test_images:
    results = model.predict(img, save=True)

In [ ]:
results=model.val()

In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])
    ))
test_img = list(uploaded.keys())[0]

In [ ]:
results = model.predict(test_img, save=True)
results[0].show()

In [ ]:
with open("requirements.txt", "w") as f:
    f.write("ultralytics\n")
    f.write("opencv-python\n")
    f.write("matplotlib\n")
    f.write("kagglehub\n")
    f.write("gradio\n")
